In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

In [ ]:
import argparse
import os
import sys
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from six.moves import cPickle
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
class model():
    def __init__(self,profile,params=None):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.y = tf.placeholder(tf.int32,[None,10])
        self.loss_counter = 0
        self.profile = profile
        self.learning_rate = 0.001
        
        if params==None:
            self.W1 = tf.get_variable(initializer=tf.random_uniform([784, 100],-0.1,0.1),name="W1")
            self.b1 = tf.get_variable(initializer=tf.zeros([100])+0.1,name="b1")
            self.W2 = tf.get_variable(initializer=tf.random_uniform([100, 100],-0.1,0.1),name="W2")
            self.b2 = tf.get_variable(initializer=tf.zeros([100])+0.1,name="b2")
            self.W3 = tf.get_variable(initializer=tf.random_uniform([100, 10],-0.1,0.1),name="W3")
            self.b3 = tf.get_variable(initializer=tf.zeros([10])+0.1,name="b3")
            self.idp = np.arange(0.1,1.05,0.1)
        else:
            self.W1 = tf.get_variable(initializer=params['W1'],name="W1")
            self.b1 = tf.get_variable(initializer=params['b1'],name="b1")
            self.W2 = tf.get_variable(initializer=params['W2'],name="W2")
            self.b2 = tf.get_variable(initializer=params['b2'],name="b2")
            self.W3 = tf.get_variable(initializer=params['W3'],name="W3")
            self.b3 = tf.get_variable(initializer=params['b3'],name="b3")
            self.idp = params['idp']
            
        def half_exp(n,k=1,dtype='float32'):
            n_ones = int(n/2)
            n_other = n - int(n/2)
            return np.append(np.ones(n_ones,dtype=dtype),np.exp((1-k)*np.arange(n_other),dtype=dtype))

        if profile == "linear":
            self.r1 = tf.get_variable(initializer=np.linspace(1,0,num=784,endpoint=False,dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.linspace(1,0,num=100,endpoint=False,dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.linspace(1,0,num=100,endpoint=False,dtype='float32') ,name="r3",dtype='float32')
        elif profile == "all-one":
            self.r1 = tf.get_variable(initializer=np.ones(784,dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.ones(100,dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.ones(100,dtype='float32'),name="r3",dtype='float32')
        elif profile == "half-exp":
            self.r1 = tf.get_variable(initializer=half_exp(784,2),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=half_exp(100,2),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=half_exp(100,2),name="r3",dtype='float32')
        else:
            self.r1 = tf.get_variable(initializer=np.array(1.0/(np.arange(784)+1),dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.array(1.0/(np.arange(100)+1),dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.array(1.0/(np.arange(100)+1),dtype='float32') ,name="r3",dtype='float32')
        
        ''' define a mask '''
        for idp in self.idp:
            with tf.variable_scope(self.profile+str(idp),reuse=None):
                n_ones = int(100 * idp)
                n_zeros = 100 - n_ones
                t2 = tf.constant(np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t2",dtype='float32')
                p2 = tf.multiply(self.r2,t2)

        ''' calculate crossentropy at different idp level and sum up as the loss '''
        loss = 0.0
        loss_list = []
        for idp in self.idp:
            z1 = tf.add(tf.matmul(self.x,self.W1),self.b1)
            y1 = tf.nn.relu(z1)
            with tf.variable_scope(self.profile+str(idp),reuse=True):
                z2 = tf.add(tf.matmul(tf.multiply(p2,y1),self.W2),self.b2)
            y2 = tf.nn.relu(z2)
            logits = tf.add(tf.matmul(y2,self.W3),self.b3)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=self.y)
            # this_loss = tf.reduce_mean(cross_entropy) * self.loss_mask[mask_counter]
            # mask_counter = mask_counter + 1
            this_loss = tf.reduce_mean(cross_entropy)
            loss = loss + this_loss
            loss_list.append(loss)

        # calculate r1*(W1*x)+b
        z1 = tf.add(tf.matmul(self.x,self.W1),self.b1)
        y1 = tf.nn.relu(z1)
        z2 = tf.add(tf.matmul(tf.multiply(self.r2,y1),self.W2),self.b2)
        y2 = tf.nn.relu(z2)
        self.logits = tf.add(tf.matmul(y2,self.W3),self.b3)
        self.probs = tf.nn.softmax(self.logits)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.y)
        # self.loss = tf.reduce_mean(cross_entropy)
        self.loss = loss_list[self.loss_counter]

        correct_prediction = tf.equal(tf.argmax(self.probs,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        tvars_trainable = tf.trainable_variables()
        # print(tvars_trainable)
        self.gamma_vars = [self.r1,self.r2,self.r3]

        for rm in self.gamma_vars:
            tvars_trainable.remove(rm)
            print('%s is not trainable.'% rm)
        self.tvars_trainable = tvars_trainable

        '''non-increasing weight clip'''
        def clip_in_order(last_output,current_input):
            added = tf.cond(current_input > last_output[0], lambda: last_output[0], lambda: current_input)
            return (added,last_output[1])
        clipped_gamma = tf.scan(fn=clip_in_order,
                                elems = self.r2,
                                initializer = (self.r2[0],self.r2[1]))
        self.gamma_clip_manually = tf.assign(ref=self.r2,value=clipped_gamma[0])

        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.tvars_trainable)
        
        self.train_op_gamma = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.gamma_vars)
    
    def train(self,sess,config,gamma_trainable=False,reuse=False,verbose=True):
        
        self.learning_rate = config['learning_rate']

        """ setting from config """
        epochs = config['epochs']
        batch_per_epoch = config['batch_per_epoch']
        batch_size = config['batch_size']
        save_dir = config['save_dir']
        mnist = config['mnist']
        input_counter = config['loss_counter']
        # loss_mask = config['loss_mask']

        """ record container """
        val_loss_per_epoch = []
        val_accu_per_epoch = []
        loss_per_epoch = []
        accu_per_epoch = []
        gamma_trainable_per_epoch = []
        
        if not reuse:
            for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
                sess.run(v.initializer)
            print("Initialized.")
            
        # print(loss_mask)

        """ training """
        for epoch in range(0,epochs):
            start_time = time.time()
            train_loss = []
            train_accu = []
            for batch in range(0,batch_per_epoch):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                if gamma_trainable:
                    _,loss, accu = sess.run([self.train_op_gamma, self.loss, self.accuracy],
                                            feed_dict = {self.x:batch_x, self.y: batch_y})
                                            # feed_dict = {self.x:batch_x, self.y: batch_y, self.loss_mask: loss_mask})
                    _ = sess.run([self.gamma_clip_manually])

                else:
                    _,loss, accu = sess.run([self.train_op, self.loss, self.accuracy], 
                                            feed_dict = {self.x:batch_x, self.y: batch_y})
                                            # feed_dict = {self.x:batch_x, self.y: batch_y, self.loss_mask: loss_mask})
                train_loss.append(loss)
                train_accu.append(accu)
            # print(sess.run(self.r2))
            loss_per_epoch.append(np.mean(train_loss))
            accu_per_epoch.append(np.mean(train_accu))
            gamma_trainable_per_epoch.append(gamma_trainable+0)

            """ validation """
            val_loss , probs, val_accu = sess.run([self.loss,self.probs,self.accuracy],
                                                  feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels})
                                                  # feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels, self.loss_mask: loss_mask})
            val_loss_per_epoch.append(val_loss)
            val_accu_per_epoch.append(val_accu)

            if verbose:
                print(" epoch: %3d \t time: %3f \t loss: %3f \t val_loss: %3f \t accu: %3f \t val_accu: %3f"% 
                  (epoch,time.time() - start_time,np.mean(train_loss), val_loss, np.mean(train_accu), val_accu))

        return({'loss':loss_per_epoch, 'val_loss':val_loss_per_epoch,
                'accu':accu_per_epoch,'val_accu':val_accu_per_epoch,
                'gamma_trainable':gamma_trainable_per_epoch})
    
    def predict(self,sess,config,idp,scope):
        with tf.variable_scope(scope,reuse=None):
            # use variable t to control the on/off of neurons
            n_ones = int(784 * idp)
            n_zeros = 784 - n_ones
            t1 = tf.get_variable(initializer=np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t1",dtype='float32')

            n_ones = int(100 * idp)
            n_zeros = 100 - n_ones
            t2 = tf.get_variable(initializer=np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t2",dtype='float32')

            n_ones = int(100 * idp)
            n_zeros = 100 - n_ones
            t3 = tf.get_variable(initializer=np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t3",dtype='float32')

            p1 = tf.get_variable(initializer=tf.multiply(self.r1,t1),name="p1")
            p2 = tf.get_variable(initializer=tf.multiply(self.r2,t2),name="p2")
            p3 = tf.get_variable(initializer=tf.multiply(self.r3,t3),name="p3")

            for v in [t1,t2,t3,p1,p2,p3]:
                sess.run(v.initializer)
            print("Running IDP mechanism {:.0f}%".format(idp*100),end='\r')

            z1 = tf.add(tf.matmul(self.x,self.W1),self.b1)
            y1 = tf.nn.relu(z1)
            # IDP on layer 2
            z2 = tf.add(tf.matmul(tf.multiply(p2,y1),self.W2),self.b2)
            y2 = tf.nn.relu(z2)
            logits = tf.add(tf.matmul(y2,self.W3),self.b3)
            self.pred_probs = tf.nn.softmax(logits)

            mnist = config['mnist']
            pred_probs = sess.run([self.pred_probs],
                                  feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels})
                                  # feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels, self.loss_mask: loss_mask})
            return(pred_probs)

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
profile_test = ["all-one","linear","harmonic","half-exp"]

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
W1 = np.random.uniform(high=0.1,low=-0.1,size=(784,100)).astype('float32')
b1 = np.random.uniform(high=0.1,low=-0.1,size=(100)).astype('float32')
W2 = np.random.uniform(high=0.1,low=-0.1,size=(100,100)).astype('float32')
b2 = np.random.uniform(high=0.1,low=-0.1,size=(100)).astype('float32')   
W3 = np.random.uniform(high=0.1,low=-0.1,size=(100,10)).astype('float32')
b3 = np.random.uniform(high=0.1,low=-0.1,size=(10)).astype('float32')
params = {'W1':W1,'W2':W2,'W3':W3,
          'b1':b1,'b2':b2,'b3':b3}

In [ ]:
profile_test = ["all-one"]

In [ ]:
counter = 0

## training using gradually complete loss function 

In [ ]:
all_epochs = 100
counter = counter + 1
profile = "all-one"
# loss_mask = np.zeros(10)
params['idp'] = np.arange(0.1,1.05,0.1)

with tf.Session() as sess:
    with tf.variable_scope(profile+str(counter)) as scope:
        intv = 0
        tidp = 0
        var_reuse = False
        go_on = True
        current_best_accu = 0
        current_best_counter = 0
        while go_on:
            # loss_mask[tidp] = 1
            save_dir = "~/IDP/output/"
            epochs = 1
            batch_per_epoch = 200
            batch_size = 28
            learning_rate = 0.001
            config = {'epochs': epochs,
                      'batch_per_epoch': batch_per_epoch,
                      'batch_size': batch_size,
                      'learning_rate': learning_rate,
                      'save_dir': save_dir,
                      'log_file': save_dir+profile+"_"+str(tidp*100)+"_log.csv",
                      'result_file': save_dir+profile+"_"+str(tidp*100)+"_result.csv",
                      'r2_file':save_dir+profile+"_"+str(tidp*100)+"_r2.csv",
                      'mnist': mnist,
                      'loss_counter':tidp
                     }
                      # 'loss_mask':loss_mask}
    
            """ result containers """
            profile_arr = []
            idp_arr = []
            accu_arr = []
            r2_dict = {}
            log = pd.DataFrame()
            
            print("============ ITERATIVE TRAINING ============")
            #for _ in range(int(all_epochs/epochs/2)):  
            intv = intv + 1
            print("Start "+str(intv)+" round")

            ''' training W,b '''
            if intv == 1:
                model_test = model(profile,params=params)
                model_test.loss_counter = tidp
                print("Start to optimize IDP = %d " % (tidp+1)*10)
                log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=var_reuse,verbose=True)
                log = pd.DataFrame.from_dict(log1)
                ''' record initial gamma of layer 2 '''
                r2_dict['initial'] = sess.run(model_test.r2)
                var_reuse = True
            else:
                log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=var_reuse,verbose=True)
                log1 = pd.DataFrame.from_dict(log1)
                log = pd.concat([log,log1])

            ''' test after W,b updates '''
            true_lab = [np.argmax(ite) for ite in mnist.test.labels]
            for idp in np.arange(0.1,1.05,0.05):  
                probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+'i'+str(int(idp*100)))
                pred_lab = [np.argmax(ite) for ite in probs[0]]
                accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                #print("IDP={:.2f}, accuracy={:.3f}".format(idp,accu))
                profile_arr.append(profile+"(train W,b in the "+str(intv)+" epochs)")
                idp_arr.append(idp)
                accu_arr.append(accu)

            ''' training gamma '''
            scope.reuse_variables()

            log1 = model_test.train(sess=sess,config=config,gamma_trainable=True,reuse=var_reuse,verbose=False)
            log1 = pd.DataFrame.from_dict(log1)
            log = pd.concat([log,log1])
            r2_dict['after_'+str(intv)] = sess.run(model_test.r2)

            ''' test after gamma updates '''
            true_lab = [np.argmax(ite) for ite in mnist.test.labels]
            this_accu = []
            for idp in np.arange(0.1,1.05,0.05):  
                probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+'i'+str(int(idp*100)))
                pred_lab = [np.argmax(ite) for ite in probs[0]]
                accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                # print("IDP={:.2f}, accuracy={:.3f}".format(idp,accu))
                profile_arr.append(profile+"(train gamma in the "+str(intv)+" epochs)")
                idp_arr.append(idp)
                accu_arr.append(accu)
                # accu list
                this_accu.append(accu)
            this_accu = np.sum(np.array(this_accu))
            if this_accu > current_best_accu:
                current_best_accu = this_accu
                current_best_counter = 0
            else:
                current_best_counter = current_best_counter + 1
                
            # if converged and wait for 3 epochs, next idp point
            if (current_best_counter >= 3) & (this_accu >= 19.0):
                tidp = tidp + 1
                current_best_counter = 0
                model_test.loss_counter = tidp
                print("Start to optimize IDP = %d " % (tidp+1)*10)
            
            # go over the whole tidp point
            if tidp == 10:
                go_on = False
            
                                      
            r2 = pd.DataFrame.from_dict(r2_dict)
            r2.to_csv(config['r2_file'],index=None)
            log.to_csv(config['log_file'],index=None)
            res = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
            res.to_csv(config['result_file'],index=None)
            print(res)

## Alternate training using different loss functions

In [ ]:
target_idp = [0.05]
all_epochs = 20
counter = counter + 1
profile = "all-one"
for tidp in target_idp:
    params['idp'] = np.array([tidp])
    interval = 1
    save_dir = "~/IDP/output/"
    epochs = interval
    batch_per_epoch = 200
    batch_size = 28
    learning_rate = 0.001
    config = {'epochs': epochs,
              'batch_per_epoch': batch_per_epoch,
              'batch_size': batch_size,
              'learning_rate': learning_rate,
              'save_dir': save_dir,
              'log_file': save_dir+profile+"_"+str(tidp*100)+"_log.csv",
              'result_file': save_dir+profile+"_"+str(tidp*100)+"_result.csv",
              'r2_file':save_dir+profile+"_"+str(tidp*100)+"_r2.csv",
              'mnist': mnist}
    print(profile)
    
    '''result containers'''
    profile_arr = []
    idp_arr = []
    accu_arr = []
    r2_dict = {}
    log = pd.DataFrame()
    with tf.Session() as sess:
        with tf.variable_scope(str(tidp)+str(counter)+str(epochs)) as scope:            
            print("============ ITERATIVE TRAINING ============")
            var_reuse = False
            for intv in range(int(all_epochs/interval/2)):  
                print("Start "+str(intv)+" round")
                '''training W,b'''
                if intv == 0:
                    model_test = model(profile,params=params)
                    log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=var_reuse,verbose=False)
                    log = pd.DataFrame.from_dict(log1)
                    ''' record initial gamma of layer 2'''
                    r2_dict['initial'] = sess.run(model_test.r2)
                    var_reuse = True
                else:
                    log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=var_reuse,verbose=False)
                    log1 = pd.DataFrame.from_dict(log1)
                    log = pd.concat([log,log1])
                
                '''test after W,b updates'''
                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+'i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    print("IDP={:.2f}, accuracy={:.3f}".format(idp,accu))
                    profile_arr.append(profile+"(train W,b in the "+str(intv)+" epochs)")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                
                '''training gamma'''
                scope.reuse_variables()
                
                log1 = model_test.train(sess=sess,config=config,gamma_trainable=True,reuse=var_reuse,verbose=False)
                log1 = pd.DataFrame.from_dict(log1)
                log = pd.concat([log,log1])
                r2_dict['after_'+str(intv)] = sess.run(model_test.r2)
                
                '''test after gamma updates'''
                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+'i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    print("IDP={:.2f}, accuracy={:.3f}".format(idp,accu))
                    profile_arr.append(profile+"(train gamma in the "+str(intv)+" epochs)")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                                  
            r2 = pd.DataFrame.from_dict(r2_dict)
            r2.to_csv(config['r2_file'],index=None)
            log.to_csv(config['log_file'],index=None)
            res = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
            res.to_csv(config['result_file'],index=None)
            print(res)

## Alternate training between gamma and weights 

In [ ]:
all_epochs = 20
counter = counter + 1
for profile in profile_test:
    save_dir = "~/IDP/output/"
    epochs = 1
    batch_per_epoch = 200
    batch_size = 28
    learning_rate = 0.001
    config = {'epochs': epochs,
              'batch_per_epoch': batch_per_epoch,
              'batch_size': batch_size,
              'learning_rate': learning_rate,
              'save_dir': save_dir,
              'log_file': save_dir+profile+"_"+str(epochs)+"_log.csv",
              'result_file': save_dir+profile+"_"+str(epochs)+"_result.csv",
              'r2_file':save_dir+profile+"_"+str(epochs)+"_r2.csv",
              'mnist': mnist}
    print(profile)
    
    '''result containers'''
    profile_arr = []
    idp_arr = []
    accu_arr = []
    r2_dict = {}
    log = pd.DataFrame()
    with tf.Session() as sess:
        with tf.variable_scope(profile+str(counter)+str(epochs)) as scope:            
            print("============ ITERATIVE TRAINING ============")
            var_reuse = False
            for intv in range(int(all_epochs/epoch/2)):  
                print("Start "+str(intv)+" round")
                '''training W,b'''
                if intv == 0:
                    model_test = model(profile,params=params)
                    log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=var_reuse,verbose=False)
                    log = pd.DataFrame.from_dict(log1)
                    ''' record initial gamma of layer 2'''
                    r2_dict['initial'] = sess.run(model_test.r2)
                    var_reuse = True
                else:
                    log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=var_reuse,verbose=False)
                    log1 = pd.DataFrame.from_dict(log1)
                    log = pd.concat([log,log1])
                
                '''test after W,b updates'''
                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+'i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    print("IDP={:.2f}, accuracy={:.3f}".format(idp,accu))
                    profile_arr.append(profile+"(train W,b in the "+str(intv)+" epochs)")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                
                '''training gamma'''
                scope.reuse_variables()
                
                log1 = model_test.train(sess=sess,config=config,gamma_trainable=True,reuse=var_reuse,verbose=False)
                log1 = pd.DataFrame.from_dict(log1)
                log = pd.concat([log,log1])
                r2_dict['after_'+str(intv)] = sess.run(model_test.r2)
                
                '''test after gamma updates'''
                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+'i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    print("IDP={:.2f}, accuracy={:.3f}".format(idp,accu))
                    profile_arr.append(profile+"(train gamma in the "+str(intv)+" epochs)")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                                  
            r2 = pd.DataFrame.from_dict(r2_dict)
            r2.to_csv(config['r2_file'],index=None)
            log.to_csv(config['log_file'],index=None)
            res = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
            res.to_csv(config['result_file'],index=None)
            print(res)

## Study on when we start training gamma

In [ ]:
epochs_test = np.array([5])
profile_test = ["linear","half-exp","all-one","harmonic"]

In [ ]:
all_epochs = 100
counter = counter + 1
for profile in profile_test:
    for epochs in epochs_test:
        save_dir = "~/IDP/output/"
        # epochs = 20
        batch_per_epoch = 200
        batch_size = 28
        learning_rate = 0.001
        config = {'epochs': epochs,
                  'batch_per_epoch': batch_per_epoch,
                  'batch_size': batch_size,
                  'learning_rate': learning_rate,
                  'save_dir': save_dir,
                  'log_file': save_dir+profile+"_"+str(epochs)+"_log.csv",
                  'result_file': save_dir+profile+"_"+str(epochs)+"_result.csv",
                  'r2_file':save_dir+profile+"_"+str(epochs)+"_r2.csv",
                  'mnist': mnist}
        print(profile)
        '''result containers'''
        profile_arr = []
        idp_arr = []
        accu_arr = []
        r2_dict = {}
        with tf.Session() as sess:
            with tf.variable_scope(profile+str(counter)+str(epochs)) as scope:
                model_test = model(profile,params=params)
                log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=False,verbose=False)
                log1 = pd.DataFrame.from_dict(log1)

                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+str(epochs)+'i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    profile_arr.append(profile+"(train W,b in the first "+str(config['epochs'])+" epochs)")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                    
                ''' record current gamma of layer 2'''
                r2_dict['before'] = sess.run(model_test.r2)
                
                print("============ FURTHER TRAINING ============")
                scope.reuse_variables()
                config['epochs'] = all_epochs - config['epochs']
                log2 = model_test.train(sess=sess,config=config,gamma_trainable=True,reuse=True,verbose=False)
                log2 = pd.DataFrame.from_dict(log2)
                log = pd.concat([log1,log2])
                log.to_csv(config['log_file'],index=None)
                #print(log)

                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope=profile+str(counter)+str(epochs)+'i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    profile_arr.append(profile+"(train gamma in the last "+str(config['epochs'])+" epochs)")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                    
                ''' record current gamma of layer 2'''
                r2_dict['after'] = sess.run(model_test.r2)
                
                r2 = pd.DataFrame.from_dict(r2_dict)
                r2.to_csv(config['r2_file'],index=None)
                
                res = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
                res.to_csv(config['result_file'],index=None)
                print(res)

## Half-half training experiment 

In [ ]:
counter = counter + 1
for profile in profile_test:
    save_dir = "~/IDP/output/"
    epochs = 1
    batch_per_epoch = 200
    batch_size = 28
    learning_rate = 0.001
    config = {'epochs': epochs,
              'batch_per_epoch': batch_per_epoch,
              'batch_size': batch_size,
              'learning_rate': learning_rate,
              'save_dir': save_dir,
              'log_file': save_dir+profile+"_log.csv",
              'result_file': save_dir+profile+"_result.csv",
              'mnist': mnist}
    print(profile)
    '''result containers'''
    profile_arr = []
    idp_arr = []
    accu_arr = []
    with tf.Session() as sess:
        with tf.variable_scope(profile+str(counter)) as scope:
            model_test = model(profile,params=params)
            log1 = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=False,verbose=False)
            log1 = pd.DataFrame.from_dict(log1)

            true_lab = [np.argmax(ite) for ite in mnist.test.labels]
            for idp in np.arange(0.1,1.05,0.05):  
                probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
                pred_lab = [np.argmax(ite) for ite in probs[0]]
                accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                profile_arr.append(profile+"(fixed gamma ep"+str(config['epochs'])+")")
                idp_arr.append(idp)
                accu_arr.append(accu)

            print("============ FURTHER TRAINING ============")
            scope.reuse_variables()

            log2 = model_test.train(sess=sess,config=config,gamma_trainable=True,reuse=True,verbose=False)
            log2 = pd.DataFrame.from_dict(log2)
            log = pd.concat([log1,log2])
            log.to_csv(config['log_file'],index=None)
            #print(log)

            true_lab = [np.argmax(ite) for ite in mnist.test.labels]
            for idp in np.arange(0.1,1.05,0.05):  
                probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
                pred_lab = [np.argmax(ite) for ite in probs[0]]
                accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                profile_arr.append(profile+"(trainable gamma ep"+str(config['epochs'])+")")
                idp_arr.append(idp)
                accu_arr.append(accu)
            #res = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
            #res.to_csv(config['result_file'],index=None)
            #print(res)
            
        print("============ TRAINING AS USUAL ============")
        
        with tf.variable_scope(profile+str(counter)+"normal"):
            model_test = model(profile=profile,params=params)
            save_dir = "~/IDP/output/"
            epochs = 80
            batch_per_epoch = 200
            batch_size = 28
            learning_rate = 0.001
            config = {'epochs': epochs,
                      'batch_per_epoch': batch_per_epoch,
                      'batch_size': batch_size,
                      'learning_rate': learning_rate,
                      'save_dir': save_dir,
                      'log_file': save_dir+profile+"_log2.csv",
                      'result_file': save_dir+profile+"_result.csv",
                      'mnist': mnist}
            with tf.Session() as sess:
                log = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=False,verbose=False)
                log = pd.DataFrame.from_dict(log)
                log.to_csv(config['log_file'],index=None)
                true_lab = [np.argmax(ite) for ite in mnist.test.labels]
                for idp in np.arange(0.1,1.05,0.05):  
                    probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
                    pred_lab = [np.argmax(ite) for ite in probs[0]]
                    accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                    profile_arr.append(profile+"(fixed gamma ep"+str(config['epochs'])+")")
                    idp_arr.append(idp)
                    accu_arr.append(accu)
                log = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
                log.to_csv(config['result_file'],index=None)
                print(log)

In [ ]:
counter = counter + 1
for profile in profile_test:
    print(profile)
    with tf.variable_scope(profile+str(counter)):
        model_test = model(profile=profile,params=params)
        save_dir = "~/IDP/output/new_loss_"
        epochs = 20
        batch_per_epoch = 200
        batch_size = 28
        learning_rate = 0.002
        config = {'epochs': epochs,
                  'batch_per_epoch': batch_per_epoch,
                  'batch_size': batch_size,
                  'learning_rate': learning_rate,
                  'save_dir': save_dir,
                  'log_file': save_dir+profile+"_log.csv",
                  'result_file': save_dir+profile+"_result.csv",
                  'mnist': mnist}
        with tf.Session() as sess:
            log = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=False,verbose=False)
            log = pd.DataFrame.from_dict(log)
            log.to_csv(config['log_file'],index=None)
            print(log)
            
            true_lab = [np.argmax(ite) for ite in mnist.test.labels]
            profile_arr = []
            idp_arr = []
            accu_arr = []
            for idp in np.arange(0.1,1.05,0.05):  
                probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
                pred_lab = [np.argmax(ite) for ite in probs[0]]
                accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                profile_arr.append(profile+"(fixed gamma ep"+str(config['epochs'])+")")
                idp_arr.append(idp)
                accu_arr.append(accu)
            log = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
            log.to_csv(config['result_file'],index=None)
            print(log)